In [1]:
import intake
import xarray as xr
import zarr
import numpy as np

import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../src/')))
#sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../../weather_routing_old/src/')))

from weather_router import isochronal_weather_router, polar, point_validity, visualize


In [2]:
#LA = (33.6, -124)
#Honolulu = (21.3, -157.8)

Cape = (-34, 18)
Rio = (-22.9, -43.2)

min_lat = min(Cape[0], Rio[0]) - 5  
max_lat = max(Cape[0], Rio[0]) + 5  

min_lon = min(Cape[1], Rio[1]) - 1  
max_lon = max(Cape[1], Rio[1]) + 1  

In [3]:

if not os.path.exists('cache.zarr/'):
    print('Downloading data...')
    ds = xr.open_zarr(
        'gs://gcp-public-data-arco-era5/ar/full_37-1h-0p25deg-chunk-1.zarr-v3',
        chunks=None,
        storage_options=dict(token='anon'),
    )
    ds = ds.rename({'latitude':'lat', 'longitude':'lon'})
    ds = ds[['10m_u_component_of_wind', '10m_v_component_of_wind']]
    ds.coords['lon'] = ((ds.coords['lon'] + 180) % 360) - 180
    ds = ds.sortby(ds.lon)
    ds = ds.sel(lat=slice(max_lat, min_lat), lon=slice(min_lon, max_lon))
    ds = ds.sel(time = slice('2025-01-01T12:00:00', '2025-01-18T12:00:00'))
    ds = ds.load()
    u10 = ds['10m_u_component_of_wind']
    v10 = ds['10m_v_component_of_wind']
    tws = xr.ufuncs.hypot(v10, u10)
    tws = tws*1.94384 #convert m/s to knots
    twd = (180 + np.rad2deg(np.arctan2(u10, v10))) % 360
    ds = tws.to_dataset(name = 'tws')
    ds['twd'] = twd
    ds.to_zarr('cache.zarr')
else:
    ds = xr.open_zarr('cache.zarr')

In [4]:
def get_wind(t, lat, lon):
    tws_sel = ds.tws.sel(time = t, method = 'nearest')
    tws_sel = tws_sel.sel(lat = lat, lon = lon, method = 'nearest')
    twd_sel = ds.twd.sel(time = t, method = 'nearest')
    twd_sel = twd_sel.sel(lat = lat, lon = lon, method = 'nearest')
    return (np.float32(twd_sel.values), np.float32(tws_sel.values))

In [5]:
volvo70_polar = polar.Polar('../test/volvo70.pol')

ds_lsm = xr.open_dataset('../src/weather_router/data/GEBCO_2025_land_mask_big_chunk.zarr').rename({'lat':'latitude', 'lon':'longitude'})
ds_lsm = ds_lsm.sortby('latitude', ascending = False)
ds_lsm = ds_lsm.fillna(0)

weatherrouter = isochronal_weather_router.weather_router(volvo70_polar, 
                                                         get_wind, 
                                                         time_steps = ds.time.values,
                                                         step = 1,
                                                         start_point = Cape,
                                                         end_point = Rio,
                                                         point_validity_file = ds_lsm,
                                                         point_validity_extent = [min_lat, min_lon, max_lat, max_lon],
                                                         spread = 130,
                                                         wake_lim = 30,
                                                         rounding = 2,
                                                         n_points=30,
                                                         )

In [6]:
weatherrouter.route()

step 1 number of isochrone points 13 dist to finish 3240.9
step 2 number of isochrone points 18 dist to finish 3228.8
step 3 number of isochrone points 20 dist to finish 3216.2
step 4 number of isochrone points 22 dist to finish 3203.6
step 5 number of isochrone points 23 dist to finish 3188.2
step 6 number of isochrone points 23 dist to finish 3173.0
step 7 number of isochrone points 22 dist to finish 3156.5
step 8 number of isochrone points 25 dist to finish 3139.3
step 9 number of isochrone points 27 dist to finish 3123.2
step 10 number of isochrone points 23 dist to finish 3107.3
step 11 number of isochrone points 27 dist to finish 3091.5
step 12 number of isochrone points 25 dist to finish 3073.5
step 13 number of isochrone points 26 dist to finish 3052.8
step 14 number of isochrone points 25 dist to finish 3031.8
step 15 number of isochrone points 22 dist to finish 3013.7
step 16 number of isochrone points 21 dist to finish 2997.2
step 17 number of isochrone points 22 dist to fin

In [7]:
route_df = weatherrouter.get_fastest_route()
route_df

,lat,lon,time,twd,tws,pos,next_pos,heading,twa,base_boat_speed,is_tacking,boat_speed,hours_elapsed,days_elapsed
0,-34.000000,18.000000,2025-01-01 12:00:00,232.094025,10.638941,"(-34.0, 18.0)","(-33.91442701217998, 17.768982280463707)",294.000000,61.905975,12.60,False,12.60,0,0.000000
1,-33.914427,17.768982,2025-01-01 13:00:00,235.547287,12.729176,"(-33.91442701217998, 17.768982280463707)","(-33.772228858767036, 17.51583059842409)",304.000000,68.452713,15.24,False,15.24,1,0.041667
2,-33.772229,17.515831,2025-01-01 14:00:00,229.544617,13.184561,"(-33.772228858767036, 17.51583059842409)","(-33.53733394471476, 17.311232239298327)",324.000000,94.455383,17.42,False,17.42,2,0.083333
3,-33.537334,17.311232,2025-01-01 15:00:00,221.543182,13.329449,"(-33.53733394471476, 17.311232239298327)","(-33.38560432044475, 17.04234458774852)",304.000000,82.456818,16.26,False,16.26,3,0.125000
4,-33.385604,17.042345,2025-01-01 16:00:00,219.955811,14.590051,"(-33.38560432044475, 17.04234458774852)","(-33.320751374990536, 16.732904045323465)",284.000000,64.044189,16.00,False,16.00,4,0.166667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
314,-22.929808,-42.500087,2025-01-14 14:00:00,117.589088,4.348788,"(-22.92980803832688, -42.50008748434434)","(-22.925928411767877, -42.51989882367333)",282.000000,164.410912,2.24,True,1.12,314,13.083333
315,-22.925928,-42.519899,2025-01-14 15:00:00,127.943924,5.807455,"(-22.925928411767877, -42.51989882367333)","(-22.990929180108857, -42.65280903671808)",242.000000,114.056076,8.32,False,8.32,315,13.125000
316,-22.990929,-42.652809,2025-01-14 16:00:00,125.464264,6.686768,"(-22.990929180108857, -42.65280903671808)","(-23.013485098112604, -42.79216674046083)",260.000000,134.535736,7.82,False,7.82,316,13.166667
317,-23.013485,-42.792167,2025-01-14 17:00:00,120.179321,6.698034,"(-23.013485098112604, -42.79216674046083)","(-23.02145295131117, -42.91672952410143)",266.000000,145.820679,6.90,False,6.90,317,13.208333


In [8]:
isochrones = weatherrouter.get_isochrones_latlon()

In [9]:
viz = visualize.visualize(ds.sel(time = slice(route_df.time.values[0], route_df.time.values[-1])), Cape, Rio, route_df, isochrone=isochrones, filename='route1')
viz.return_plot()

:DynamicMap   [time]
   :Overlay
      .WMTS.I        :WMTS   [Longitude,Latitude]
      .Image.I       :Image   [lon,lat]   (tws)
      .VectorField.I :VectorField   [lon,lat]   (twd_rad,tws)
      .Points.I      :Points   [lon,lat]
      .Points.II     :Points   [lon,lat]
      .Path.I        :Path   [Longitude,Latitude]
      .Points.III    :Points   [lon,lat]   (TWS,TWD,TWA,Boat Speed)
      .Points.IV     :Points   [Longitude,Latitude]

In [10]:
initial_route = weatherrouter.get_fastest_route(stats=False)
initial_isochrones = weatherrouter.get_isochrones()

optimized_route = weatherrouter.optimize(initial_route, initial_isochrones)

In [11]:
route_df = weatherrouter.get_fastest_route(use_optimized = True)
route_df

,lat,lon,time,twd,tws,pos,next_pos,heading,twa,base_boat_speed,is_tacking,boat_speed,hours_elapsed,days_elapsed
0,-34.000000,18.000000,2025-01-01 12:00:00,232.094025,10.638941,"(-34.0, 18.0)","(-33.91442701217998, 17.768982280463707)",294.000000,61.905975,12.60,False,12.60,0,0.000000
1,-33.914427,17.768982,2025-01-01 13:00:00,235.547287,12.729176,"(-33.91442701217998, 17.768982280463707)","(-33.772228858767036, 17.51583059842409)",304.000000,68.452713,15.24,False,15.24,1,0.041667
2,-33.772229,17.515831,2025-01-01 14:00:00,229.544617,13.184561,"(-33.772228858767036, 17.51583059842409)","(-33.53733394471476, 17.311232239298327)",324.000000,94.455383,17.42,False,17.42,2,0.083333
3,-33.537334,17.311232,2025-01-01 15:00:00,221.543182,13.329449,"(-33.53733394471476, 17.311232239298327)","(-33.38560432044475, 17.04234458774852)",304.000000,82.456818,16.26,False,16.26,3,0.125000
4,-33.385604,17.042345,2025-01-01 16:00:00,219.955811,14.590051,"(-33.38560432044475, 17.04234458774852)","(-33.320751374990536, 16.732904045323465)",284.000000,64.044189,16.00,False,16.00,4,0.166667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
314,-22.929808,-42.500087,2025-01-14 14:00:00,117.589088,4.348788,"(-22.92980803832688, -42.50008748434434)","(-22.925928411767877, -42.51989882367333)",282.000000,164.410912,2.24,True,1.12,314,13.083333
315,-22.925928,-42.519899,2025-01-14 15:00:00,127.943924,5.807455,"(-22.925928411767877, -42.51989882367333)","(-22.990929180108857, -42.65280903671808)",242.000000,114.056076,8.32,False,8.32,315,13.125000
316,-22.990929,-42.652809,2025-01-14 16:00:00,125.464264,6.686768,"(-22.990929180108857, -42.65280903671808)","(-23.013485098112604, -42.79216674046083)",260.000000,134.535736,7.82,False,7.82,316,13.166667
317,-23.013485,-42.792167,2025-01-14 17:00:00,120.179321,6.698034,"(-23.013485098112604, -42.79216674046083)","(-23.02145295131117, -42.91672952410143)",266.000000,145.820679,6.90,False,6.90,317,13.208333


In [12]:
restricted_isochrones = weatherrouter.get_optimized_isochrones_latlon()

In [13]:
viz = visualize.visualize(ds.sel(time = slice(route_df.time.values[0], route_df.time.values[-1])), Cape, Rio, route_df, isochrone=restricted_isochrones, filename='route1')
viz.return_plot()

:DynamicMap   [time]
   :Overlay
      .WMTS.I        :WMTS   [Longitude,Latitude]
      .Image.I       :Image   [lon,lat]   (tws)
      .VectorField.I :VectorField   [lon,lat]   (twd_rad,tws)
      .Points.I      :Points   [lon,lat]
      .Points.II     :Points   [lon,lat]
      .Path.I        :Path   [Longitude,Latitude]
      .Points.III    :Points   [lon,lat]   (TWS,TWD,TWA,Boat Speed)

In [14]:
ds_lsm = xr.open_dataset('s3://peterm790/GEBCO_2025_land_mask_big_chunk.zarr/',
                            engine = 'zarr',
                            storage_options={"anon": True})

ImportError: Install s3fs to access S3

In [16]:
import s3fs

fs = s3fs.S3FileSystem(anon=True)
fs.ls("peterm790/GEBCO_2025_land_mask_big_chunk.zarr")


['peterm790/GEBCO_2025_land_mask_big_chunk.zarr/lat',
 'peterm790/GEBCO_2025_land_mask_big_chunk.zarr/lon',
 'peterm790/GEBCO_2025_land_mask_big_chunk.zarr/lsm']